# [Snowflake ML](https://www.snowflake.com/en/data-cloud/snowflake-ml/)
Conjunto integrado de capacidades endo-to-end para ML en una única plataforma sobre sus datos gobernados. 
Los científicos de datos e ingenieros de ML pueden desarrollar y poner en producción características y modelos escalables de manera fácil y segura sin movimiento de datos, silos o compromisos de gobernanza. La librería Snowpark ML Python (el paquete snowflake-ml-python) proporciona APIs para desarrollar e implementar sus pipelines de Snowflake ML.

Esta es la parte 3 de una serie de inicio rápido de introducción de 3 partes a Snowflake Feature Store (vea la parte 1 [aquí](https://quickstarts.snowflake.com/guide/intro-to-feature-store/index.html#0) y la parte 2 [aquí](https://quickstarts.snowflake.com/guide/overview-of-feature-store-api/index.html?index=..%2F..index#0)). Este inicio rápido demuestra un ciclo completo de experimento de ML de principio a fin, incluyendo la creación de características, la generación de datos de entrenamiento, el entrenamiento del modelo y la inferencia. El flujo de trabajo aborda características clave de Snowflake ML, incluyendo [Snowflake Feature Store](https://docs.snowflake.com/en/developer-guide/snowpark-ml/feature-store/overview), [Dataset](https://docs.snowflake.com/en/developer-guide/snowpark-ml/dataset), [APIs de Snowflake ML](https://docs.snowflake.com/en/developer-guide/snowpark-ml/modeling) y [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/overview).


[Fuente](https://quickstarts.snowflake.com/guide/develop-and-manage-ml-models-with-feature-store-and-model-registry/index.html?index=..%2F..index#0)

# Desarrollar y Administrar Modelos ML con Feature Store y Model Registry

Este notebook muestra un ciclo completo de experimentación de ML de principio a fin, incluyendo la creación de características, la generación de datos de entrenamiento, el entrenamiento del modelo y la inferencia. El flujo de trabajo aborda características clave de Snowflake ML, incluyendo [Snowflake Feature Store](https://docs.snowflake.com/en/developer-guide/snowpark-ml/feature-store/overview), [Dataset](https://docs.snowflake.com/en/developer-guide/snowpark-ml/dataset), [Snowpark ML Modeling](https://docs.snowflake.com/en/developer-guide/snowpark-ml/modeling) y [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/overview).

Nota: Asegúrese de que `snowflake-ml-python` y `snowflake-snowpark-python` estén instalados desde el menú desplegable de paquetes.

**Tabla de contenidos**
- Configurar entorno de prueba
  - Conectar a Snowflake
  - Seleccionar su ejemplo
- Crear características con Feature Store
  - Inicializar Feature Store
  - Registrar entidades y vistas de características
  - Examinar características en la UI de Snowflake
- Generar Datos de Entrenamiento
- Entrenar modelo con Snowpark ML
- Registrar modelos en Model Registry
  - Examinar modelo en la UI de Snowflake
- Predecir con el modelo
  - Predecir con modelo local
  - Predecir con Model Registry


## Configurar entorno de prueba

### Conectar a Snowflake

Comencemos configurando nuestro entorno de prueba

In [1]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", "name":"aiml_notebooks_develop_models_with_feature_store", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":1, "source":"notebook"}}

# Set session context 
# session.use_role("ML_MODEL_ROLE") 

# Print the current role, warehouse, and database/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

import warnings
warnings.filterwarnings('ignore')

In [2]:
# The schema where Feature Store will initialize on and test dataset stores.
FS_DEMO_SCHEMA = session.get_current_schema()
# the schema where the model lives.
MODEL_DEMO_SCHEMA = session.get_current_schema()

[Row(status='Schema SNOWFLAKE_FEATURE_STORE_NOTEBOOK_DEMO successfully created.')]

### Seleccionar su ejemplo

Hemos preparado algunos ejemplos que puede encontrar en nuestro [repositorio de código abierto](https://github.com/snowflakedb/snowflake-ml-python/tree/main/snowflake/ml/feature_store/examples). Cada ejemplo contiene el conjunto de datos de origen, la vista de características y las definiciones de entidad que se utilizarán en esta demostración. `ExampleHelper` (incluido en snowflake-ml-python) configurará todo con APIs simples y no tendrá que preocuparse por los detalles.

In [3]:
from snowflake.ml.feature_store.examples.example_helper import ExampleHelper

example_helper = ExampleHelper(session, session.get_current_database(), FS_DEMO_SCHEMA)
example_helper.list_examples().to_pandas()

,NAME,DESC,LABEL_COLS
0,new_york_taxi_features,Features using taxi trip data trying to predic...,TOTAL_AMOUNT
1,airline_features,Features using synthetic airline data to predi...,DEPARTING_DELAY
2,citibike_trip_features,Features using citibike trip data trying to pr...,tripduration
3,wine_quality_features,Features using wine quality data trying to pre...,quality


`load_example()` cargará los datos de origen en tablas de Snowflake. En el ejemplo siguiente, estamos utilizando el ejemplo "new_york_taxi_features". Puede reemplazarlo con cualquier ejemplo listado anteriormente. La ejecución de la celda siguiente puede tardar algún tiempo dependiendo del tamaño del conjunto de datos.

In [4]:
# replace the value with the example you want to run
source_tables = example_helper.load_example('new_york_taxi_features')

# display as Pandas DataFrame
for table in source_tables:
    print(f"{table}:")
    df = session.table(table).limit(5).to_pandas()
    df.style

"AIRLINE_FEATURE_STORE".SNOWFLAKE_FEATURE_STORE_NOTEBOOK_DEMO.nyc_yellow_trips:


,VENDORID,PASSENGER_COUNT,TRIP_DISTANCE,RATECODEID,STORE_AND_FWD_FLAG,PULOCATIONID,DOLOCATIONID,PAYMENT_TYPE,FARE_AMOUNT,EXTRA,MTA_TAX,TIP_AMOUNT,TOLLS_AMOUNT,IMPROVEMENT_SURCHARGE,TOTAL_AMOUNT,CONGESTION_SURCHARGE,AIRPORT_FEE,TPEP_PICKUP_DATETIME,TPEP_DROPOFF_DATETIME,TRIP_ID
0,2,1,0.65,1,N,238,238,2,4.0,1.0,0.5,0.0,0.0,0.3,5.8,NaN,NaN,2016-01-07 17:53:47,2016-01-07 17:55:29,2195456
1,2,1,1.24,1,N,138,70,2,7.5,1.0,0.5,0.0,0.0,0.3,9.3,NaN,NaN,2016-01-07 17:37:22,2016-01-07 17:46:17,2195457
2,1,1,0.90,1,N,161,229,2,6.5,1.0,0.5,0.0,0.0,0.3,8.3,NaN,NaN,2016-01-07 17:08:42,2016-01-07 17:16:12,2195458
3,1,1,2.50,1,N,162,262,1,12.0,1.0,0.5,1.0,0.0,0.3,14.8,NaN,NaN,2016-01-07 17:27:16,2016-01-07 17:42:22,2195459
4,1,1,1.20,1,N,262,141,2,8.0,1.0,0.5,0.0,0.0,0.3,9.8,NaN,NaN,2016-01-07 17:46:03,2016-01-07 17:56:03,2195460


## Crear características con Feature Store

### Inicializar Feature Store

Primero, creemos un cliente de feature store. Con el modo `CREATE_IF_NOT_EXIST`, intentará crear un nuevo esquema de Feature Store y todos los metadatos necesarios de feature store si aún no existen. Es necesario por primera vez configurar un Feature Store. Después, puede usar el modo `FAIL_IF_NOT_EXIST` para conectarse a un Feature Store existente.

Tenga en cuenta que la base de datos que se está utilizando ya debe existir. Feature Store **NO** intentará crear la base de datos incluso en el modo `CREATE_IF_NOT_EXIST`.

In [5]:
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)

fs = FeatureStore(
    session=session, 
    database=session.get_current_database(), 
    name=FS_DEMO_SCHEMA, 
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

### Registrar entidades y vistas de características

A continuación, registramos nuevas entidades y vistas de características en Feature Store. Las entidades serán las claves de unión utilizadas para generar datos de entrenamiento. Las vistas de características contienen todas las características que necesita para el entrenamiento y la inferencia de su modelo. Tenemos entidades y vistas de características para este ejemplo definidas en nuestro [repositorio de código abierto](https://github.com/snowflakedb/snowflake-ml-python/tree/main/snowflake/ml/feature_store/examples). Cargaremos las definiciones con `load_entities()` y `load_draft_feature_views()` para simplificar.

In [6]:
all_entities = []
for e in example_helper.load_entities():
    entity = fs.register_entity(e)
    all_entities.append(entity)
fs.list_entities().show()

----------------------------------------------------------------------
|"NAME"        |"JOIN_KEYS"       |"DESC"                 |"OWNER"   |
----------------------------------------------------------------------
|DOLOCATIONID  |["DOLOCATIONID"]  |Drop off location id.  |ENGINEER  |
|TRIP_ID       |["TRIP_ID"]       |Trip id.               |ENGINEER  |
----------------------------------------------------------------------



In [7]:
all_feature_views = []
for fv in example_helper.load_draft_feature_views():
    rf = fs.register_feature_view(
        feature_view=fv,
        version='1.0'
    )
    all_feature_views.append(rf)

fs.list_feature_views().select('name', 'version', 'desc', 'refresh_freq').show()

------------------------------------------------------------------------------------------------
|"NAME"      |"VERSION"  |"DESC"                                              |"REFRESH_FREQ"  |
------------------------------------------------------------------------------------------------
|F_LOCATION  |1.0        |Features aggregated by location id and refreshe...  |12 hours        |
|F_TRIP      |1.0        |Features per trip refreshed every day.              |1 day           |
------------------------------------------------------------------------------------------------



Podemos examinar todas las características en una vista de características.

In [8]:
import pandas as pd

for fv in all_feature_views:
    print(f"{fv.name}/{fv.version} has features:")
    pd.DataFrame(fv.feature_descs.items(), columns=['Name', 'Desc']).style

F_LOCATION/1.0 has features:


,Name,Desc
0,AVG_FARE_1H,Averaged fare in past 1 hour window aggregated...
1,AVG_FARE_10H,Averaged fare in past 10 hours aggregated by l...


F_TRIP/1.0 has features:


,Name,Desc
0,PASSENGER_COUNT,The count of passenger of a trip.
1,TRIP_DISTANCE,The distance of a trip.
2,FARE_AMOUNT,The fare of a trip.


### Examinar características en la UI de Snowflake
Ahora debería poder ver las entidades y vistas de características registradas en la UI de Snowflake.

In [ ]:
import streamlit as st
st.image("https://raw.githubusercontent.com/Snowflake-Labs/sfguide-develop-and-manage-ml-models-with-feature-store-and-model-registry/refs/heads/main/notebooks/feature-store-ui.png")

## Generar Datos de Entrenamiento

Una vez que nuestros pipelines de características estén completamente configurados, podemos usarlos para generar [Snowflake Dataset](https://docs.snowflake.com/en/developer-guide/snowpark-ml/dataset) y posteriormente realizar el entrenamiento del modelo. Generar datos de entrenamiento es fácil ya que las FeatureViews materializadas ya llevan la mayoría de los metadatos como claves de unión, marca de tiempo para búsquedas puntuales, etc. Solo necesitamos proporcionar los datos "spine" (se llama spine porque es la lista de IDs de entidad que estamos enriqueciendo esencialmente uniendo características con ellos).

`generate_dataset()` devuelve un objeto Snowflake Dataset, que es óptimo para el entrenamiento distribuido con frameworks de aprendizaje profundo como TensorFlow o Pytorch que requieren acceso a nivel de archivo detallado. Crea un nuevo objeto Dataset (que está versionado e inmutable) en Snowflake que materializa los datos en archivos Parquet. Si entrena modelos con librerías clásicas de ML como Snowpark ML o scikit-learn, puede usar `generate_training_set()` que devuelve una tabla clásica de Snowflake. La celda siguiente demuestra `generate_dataset()`.

Recuperar algunas columnas de metadatos que son esenciales al generar datos de entrenamiento.

In [9]:
label_cols = example_helper.get_label_cols()
timestamp_col = example_helper.get_training_data_timestamp_col()
excluded_cols = example_helper.get_excluded_cols()
join_keys = [key for entity in all_entities for key in entity.join_keys]
spine_table = example_helper.get_training_spine_table()
print(f'timestamp col: {timestamp_col}')
print(f'excluded cols: {excluded_cols}')
print(f'label cols: {label_cols}')
print(f'join keys: {join_keys}')
print(f'training spine table: {spine_table}')

timestamp col: TPEP_PICKUP_DATETIME
excluded cols: []
label cols: ['TOTAL_AMOUNT']
join keys: ['TRIP_ID', 'DOLOCATIONID']
training spine table: "AIRLINE_FEATURE_STORE".SNOWFLAKE_FEATURE_STORE_NOTEBOOK_DEMO.nyc_yellow_trips


Crear un dataframe spine que se muestree apartir de la tabla de origen.

In [10]:
sample_count = 512
source_df = session.sql(f"""
    select {','.join(label_cols)}, 
            {','.join(join_keys)} 
            {',' + timestamp_col if timestamp_col is not None else ''} 
    from {spine_table}
""")
spine_df = source_df.sample(n=sample_count)
# preview spine dataframe
spine_df.show()

------------------------------------------------------------------------
|"TOTAL_AMOUNT"  |"TRIP_ID"  |"DOLOCATIONID"  |"TPEP_PICKUP_DATETIME"  |
------------------------------------------------------------------------
|11.8            |4391772    |236             |2016-01-13 15:28:31     |
|6.8             |9640580    |231             |2016-01-28 21:47:03     |
|10.3            |8986296    |162             |2016-01-27 06:44:50     |
|20.35           |4689446    |261             |2016-01-14 09:29:27     |
|19.89           |9360850    |166             |2016-01-28 07:33:07     |
|6.3             |9335036    |211             |2016-01-28 04:46:46     |
|72.92           |5223446    |264             |2016-01-15 17:21:27     |
|16.3            |4578405    |116             |2016-01-13 23:35:00     |
|7.3             |5045083    |163             |2016-01-15 07:10:06     |
|10.3            |9733135    |145             |2016-01-29 05:14:06     |
---------------------------------------------------

Generar objeto dataset a partir del dataframe spine y las vistas de características.

In [11]:
my_dataset = fs.generate_dataset(
    name="my_cool_training_dataset",
    spine_df=spine_df, 
    features=all_feature_views,
    version="4.0",
    spine_timestamp_col=timestamp_col,
    spine_label_cols=label_cols,
    exclude_columns=excluded_cols,
    desc="This is the dataset joined spine dataframe with feature views",
)

Convierta el dataset a un dataframe de Snowpark y examine todas las características en él.

In [12]:
training_data_df = my_dataset.read.to_snowpark_dataframe()
assert training_data_df.count() == sample_count
# drop rows that have any nulls in value. 
training_data_df = training_data_df.dropna(how='any')
training_data_df.to_pandas()

,TOTAL_AMOUNT,TRIP_ID,DOLOCATIONID,TPEP_PICKUP_DATETIME,AVG_FARE_1H,AVG_FARE_10H,PASSENGER_COUNT,TRIP_DISTANCE,FARE_AMOUNT
0,6.300000,9753533,228,2016-01-29 07:50:17-08:00,42.083332,25.560465,1,0.800000,5.5
1,-4.800000,337386,161,2016-01-01 23:27:54-08:00,9.713513,10.708535,2,0.150000,-3.5
2,12.250000,521354,161,2016-01-02 16:00:37-08:00,10.513055,9.510478,3,1.110000,9.0
3,18.799999,536727,161,2016-01-02 17:30:49-08:00,11.397975,9.790948,5,2.500000,16.0
4,15.300000,1059372,161,2016-01-04 11:12:42-08:00,10.893401,9.468452,1,2.500000,12.5
...,...,...,...,...,...,...,...,...,...
507,38.299999,371165,244,2016-01-02 02:56:18-08:00,23.259615,22.674618,1,13.300000,37.0
508,21.799999,4212437,244,2016-01-13 04:03:58-08:00,22.120001,22.395477,1,7.170000,20.5
509,42.349998,7530490,244,2016-01-21 17:51:06-08:00,25.574074,23.103773,1,7.900000,33.5
510,14.300000,7304440,4,2016-01-21 03:12:42-08:00,8.818182,10.719931,1,2.680000,12.0


## Entrenar modelo con Snowpark ML

Ahora entrenemos un modelo simple de random forest y evaluemos la precisión de la predicción. Cuando llama a fit() en un DataFrame que se crea a partir de un Dataset, la vinculación entre el modelo entrenado y el dataset se configura automáticamente.

In [13]:
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling import metrics as snowml_metrics
from snowflake.snowpark.functions import abs as sp_abs, mean, col

def train_model_using_snowpark_ml(training_data_df):
    train, test = training_data_df.random_split([0.8, 0.2], seed=42)
    feature_columns = list(set(training_data_df.columns) - set(label_cols) - set(join_keys) - set([timestamp_col]))
    print(f"feature cols: {feature_columns}")
    
    rf = RandomForestRegressor(
        input_cols=feature_columns, label_cols=label_cols, 
        max_depth=3, n_estimators=20, random_state=42
    )

    rf.fit(train)
    predictions = rf.predict(test)

    output_label_names = ['OUTPUT_' + col for col in label_cols]
    mse = snowml_metrics.mean_squared_error(
        df=predictions, 
        y_true_col_names=label_cols, 
        y_pred_col_names=output_label_names
    )

    accuracy = 100 - snowml_metrics.mean_absolute_percentage_error(
        df=predictions,
        y_true_col_names=label_cols,
        y_pred_col_names=output_label_names
    )

    print(f"MSE: {mse}, Accuracy: {accuracy}")
    return rf

random_forest_model = train_model_using_snowpark_ml(training_data_df) 

feature cols: ['TRIP_DISTANCE', 'FARE_AMOUNT', 'AVG_FARE_10H', 'PASSENGER_COUNT', 'AVG_FARE_1H']
MSE: 8.587654420611477, Accuracy: 99.83667856616516


## Registrar modelo en Model Registry

Después de entrenar el modelo, podemos guardarlo en Model Registry para poder administrar el modelo, sus metadatos incluyendo métricas, versiones, y usarlo posteriormente para inferencia.

In [14]:
from snowflake.ml.registry import Registry

registry = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=MODEL_DEMO_SCHEMA,
)

Registrar modelo en Model Registry.

In [15]:
model_name = "MY_RANDOM_FOREST_REGRESSOR_MODEL"

registry.log_model(
    model_name=model_name,
    version_name="v1",
    model=random_forest_model,
    comment="My model trained with feature views, dataset",
)

/Users/wezhou/miniconda3/envs/py38/lib/python3.8/contextlib.py:113: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)


ModelVersion(
  name='MY_RANDOM_FOREST_REGRESSOR_MODEL',
  version='V1',
)

### Examinar modelo en la UI de Snowflake
Ahora debería poder ver el modelo en la UI de Snowflake.

In [ ]:
import streamlit as st
st.image("https://raw.githubusercontent.com/Snowflake-Labs/sfguide-develop-and-manage-ml-models-with-feature-store-and-model-registry/refs/heads/main/notebooks/model-registry-ui.png")

## Predecir con el modelo

¡Finalmente, estamos casi listos para la predicción! Para esto, podemos buscar los valores de características más recientes en Feature Store para los registros de datos específicos sobre los que estamos ejecutando la predicción. Uno de los beneficios clave de usar Feature Store es que proporciona una forma de servir automáticamente los valores de características correctos durante la predicción con valores de características correctos en un punto en el tiempo. `load_feature_views_from_dataset()` obtiene las mismas vistas de características utilizadas en el entrenamiento, luego `retrieve_feature_values()` busca los valores de características más recientes.

In [19]:
test_df = source_df.sample(n=3)

# load back feature views from dataset
fvs = fs.load_feature_views_from_dataset(my_dataset)
enriched_df = fs.retrieve_feature_values(
    test_df, 
    features=fvs,
    exclude_columns=join_keys,
    spine_timestamp_col=timestamp_col
)
enriched_df = enriched_df.drop(join_keys)
enriched_pd = enriched_df.to_pandas()

### [Opcional 1] predecir con modelo local
Ahora podemos predecir con un modelo local y los valores de características recuperados de feature store.

In [20]:
pred = random_forest_model.predict(enriched_pd)
print(pred)

   TOTAL_AMOUNT TPEP_PICKUP_DATETIME  AVG_FARE_1H  AVG_FARE_10H  \
0         15.96  2016-01-07 10:26:02     9.440415      9.324965   
1         10.55  2016-01-01 18:44:40    10.083333      9.236685   
2         17.80  2016-01-29 21:05:54    10.385390     10.287410   

   PASSENGER_COUNT  TRIP_DISTANCE  FARE_AMOUNT  OUTPUT_TOTAL_AMOUNT  
0                1           2.23         12.5            16.440312  
1                1           1.70          7.0             8.523669  
2                1           3.11         16.5            18.717726  


### [Opción 2] Predecir con Model Registry

También podemos recuperar el modelo de model registry y ejecutar predicciones sobre el modelo utilizando los valores de características más recientes.

In [21]:
# model is retrieved from Model Registry
model = registry.get_model(model_name).version("v1")
restored_prediction = model.run(enriched_pd, function_name="predict")
print(restored_prediction)

   TRIP_DISTANCE  FARE_AMOUNT  AVG_FARE_10H  PASSENGER_COUNT  AVG_FARE_1H  \
0           2.23         12.5      9.324965                1     9.440415   
1           1.70          7.0      9.236685                1    10.083333   
2           3.11         16.5     10.287410                1    10.385390   

   OUTPUT_TOTAL_AMOUNT  
0            16.440312  
1             8.523669  
2            18.717726  
